# Recognition

In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import time
detector = HandDetector(maxHands=1)
classifier = Classifier("Model/keras_model.h5", "Model/labels.txt")
offset = 20
imgSize = 300
# folder = "Rock"
# folder = "Paper"
# folder = "Scissor"


labels = ["R", "P", "S"]

In [2]:
def recognition(cap,detector,classifier,offset,imgSize,labels):
    while True:
        success, img = cap.read()
        imgOutput = img.copy()
        hands, img = detector.findHands(img)
        if hands:
            hand = hands[0]
            x, y, w, h = hand['bbox']

            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
            imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

            imgCropShape = imgCrop.shape

            aspectRatio = h / w

            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                imgResizeShape = imgResize.shape
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
                prediction, index = classifier.getPrediction(imgWhite, draw=False)
                # print(prediction, index)

            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                imgResizeShape = imgResize.shape
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize
                prediction, index = classifier.getPrediction(imgWhite, draw=False)
            cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                          (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
            cv2.putText(imgOutput, labels[index], (x, y - 26),
                        cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
            cv2.rectangle(imgOutput, (x-offset, y-offset),
                          (x + w+offset, y + h+offset), (255, 0, 255), 4)

            cv2.imshow("ImageCrop", imgCrop)
            cv2.imshow("ImageWhite", imgWhite)

        cv2.imshow("Image", imgOutput)
        key=cv2.waitKey(1)

        if key == ord("s"):
            return index 

# RL part

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import time

In [4]:
from pyfirmata import Arduino, SERVO, util
from time import sleep

port1 = 'COM10'

pin1 = 10
pin2 = 6

board1 = Arduino(port1)

board1.digital[pin1].mode = SERVO
board1.digital[pin2].mode = SERVO
board1.digital[pin1].write(180)
def rotate_servo1(pin1, angle):
    
    board1.digital[pin1].write(angle)
    
    
    sleep(1)
def rotate_servo2(pin2, angle):

    board1.digital[pin2].write(angle)

    sleep(1)
def rock_(pin1,pin2,angle1,angle2):
    board1.digital[pin1].write(angle1)
    board1.digital[pin2].write(angle2)
    sleep(1)

In [ ]:
class Bot(object):
    # our states can be either "ROCK, PAPER or SCISSORS"
    state_space = 3
    # three actions by our player
    action_space = 3
    q_table = np.random.uniform(low = -2, high = 5, size = (3, 3))
    total_reward, reward = 0,0
    total_reward, reward = 0,0
    total_wincount,wincount=0,0
    total_tiecount,tiecount=0,0
    total_lostcount,lostcount=0,0
    avg_rewards_list = []
    avg_reward = 0
    result = 'DRAW'
    tags = ["ROCK", "PAPER", "SCISSORS"]
    # looses to map
    loses_to = {
       "0": 1, # rock loses to paper
       "1": 2,  # paper loses to scissor
       "2": 0  # scissor loses to rock
    }

    def __init__(self, alpha=0.5, gamma=0.2, epsilon=0.8, min_eps=0, episodes=50):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.min_eps = min_eps
        self.episodes = episodes
        # Calculate episodic reduction in epsilon
        self.reduction = (epsilon - min_eps) / episodes

    # either explore or exploit, any which ways return the next action
    def bot_move(self, player_move):
        action = 0
        # Determine next action - epsilon greedy strategy
        if np.random.random() < 1 - self.epsilon:
            print("Exploiting....")
            action = np.argmax(self.q_table[player_move])
        else:
            print("Exploring.....")
            action = np.random.randint(0, self.action_space)
        # Decay epsilon
        if self.epsilon > self.min_eps:
            self.epsilon -= self.reduction
        print("choose ",self.tags[action])
        return action

    def get_reward(self, player, bot):
        reward = 0
        wincount=0
        tiecount=0
        lostcount=0
        if self.get_result(player, bot) == 'WIN':
            reward = 1
            wincount=1

        elif self.get_result(player, bot) == 'LOSE':
            reward = -1
            lostcount=1

        else:
            # Draw case
            reward = 0
            tiecount=1

        return reward,wincount,tiecount,lostcount

    # update q_table
    def update_experience(self, state, action, reward):
        delta = self.alpha * (reward + self.gamma * np.max(self.q_table[action]) - self.q_table[state,action])
        self.q_table[state,action] += delta

    def print_stats(self, player, bot, reward):
        print("Player move : {0}, bot: {1}, reward: {2}, result: {3}, total_reward: {4}".format(self.tags[player], self.tags[bot], reward, self.result, self.total_reward))
        print(self.q_table)
        print("Total Win count:",self.total_wincount,"||","Win rate:",(self.total_wincount/200)*100)
        print("Total Tie count:",self.total_tiecount,"||","Tie rate:",(self.total_tiecount/200)*100)
        print("Total Lost count:",self.total_lostcount,"||","Lost rate:",(self.total_lostcount/200)*100)
        pass
    # returns either a WIN, LOSE or a DRAW to indicate the same.
    def get_result(self, player_move, bot_move):
        if bot_move == player_move:
            self.result = 'DRAW'
        elif self.loses_to[str(bot_move)] == player_move :
            self.result = 'LOSE'
        else:
            self.result = 'WIN'

        return self.result    

    def get_avg_rewards(self):
        return self.avg_rewards_list

    def play(self, player_move):
        # add reward
        bot_move = self.bot_move(player_move)
        if bot_move == 0:
            rock_(pin1,pin2,0,180)
            sleep(1)
            rock_(pin1,pin2,180,0)
        elif bot_move==1:
            pass
        else:
            rotate_servo1(pin2, 180)
            sleep(1)
            rotate_servo1(pin2, 0)
            
        reward,wincount,tiecount,lostcount = self.get_reward(player_move, bot_move)
        self.total_reward += reward
        self.total_wincount+=wincount
        self.total_tiecount+=tiecount
        self.total_lostcount+=lostcount
        self.avg_rewards_list.append(reward)
        # update experience
        self.update_experience(player_move, bot_move, reward)
        self.print_stats(player_move, bot_move, reward)

class Game(object):

    def __init__(self, bot, episodes=200):
        self.bot = bot
        self.episodes = episodes

    def begin(self):
        #player_moves = self.generate_fake_moves(True)
        cap = cv2.VideoCapture(0)
        for idx in range(0, self.episodes):
            #player_moves=int(input("Enter your move (0- Rock, 1 - paper, 2- scissors): "))
            player_moves=recognition(cap,detector,classifier,offset,imgSize,labels)
            self.bot.play(player_moves)
        # Plot Rewards
#         plt.plot(np.arange(len(self.bot.get_avg_rewards())) + 1)
#         plt.savefig('rewards.png')     
#         plt.close()
    def generate_fake_moves(self, fake_moves):
        player_moves = []
        for i in range(0, self.episodes):
            if fake_moves:
                player_moves.append(np.random.randint(0, 3))
            else:
                self.read_player_move()
        return player_moves

#     def read_player_move(self):
#         return int(input("Enter your move (0- Rock, 1 - paper, 2- scissors): "))

game = Game(Bot())
game.begin()

Exploring.....
choose  ROCK
Player move : SCISSORS, bot: ROCK, reward: 1, result: WIN, total_reward: 1
[[ 0.74268912  1.01334528 -0.54069428]
 [ 3.93256332  1.02324988 -1.72998998]
 [ 2.28637206 -0.20004544  4.3225316 ]]
Total Win count: 1 || Win rate: 0.5
Total Tie count: 0 || Tie rate: 0.0
Total Lost count: 0 || Lost rate: 0.0
Exploiting....
choose  ROCK
Player move : PAPER, bot: ROCK, reward: -1, result: LOSE, total_reward: 0
[[ 0.74268912  1.01334528 -0.54069428]
 [ 1.56761619  1.02324988 -1.72998998]
 [ 2.28637206 -0.20004544  4.3225316 ]]
Total Win count: 1 || Win rate: 0.5
Total Tie count: 0 || Tie rate: 0.0
Total Lost count: 1 || Lost rate: 0.5
Exploiting....
choose  SCISSORS
Player move : SCISSORS, bot: SCISSORS, reward: 0, result: DRAW, total_reward: 0
[[ 0.74268912  1.01334528 -0.54069428]
 [ 1.56761619  1.02324988 -1.72998998]
 [ 2.28637206 -0.20004544  2.59351896]]
Total Win count: 1 || Win rate: 0.5
Total Tie count: 1 || Tie rate: 0.5
Total Lost count: 1 || Lost rate: 0.5

Exploring.....
choose  PAPER
Player move : PAPER, bot: PAPER, reward: 0, result: DRAW, total_reward: 3
[[ 0.74268912  1.01334528 -0.54069428]
 [-0.64952175  0.22006248  1.09551494]
 [ 1.23653477 -0.71201187  0.45916616]]
Total Win count: 10 || Win rate: 5.0
Total Tie count: 9 || Tie rate: 4.5
Total Lost count: 7 || Lost rate: 3.5000000000000004
Exploiting....
choose  ROCK
Player move : SCISSORS, bot: ROCK, reward: 1, result: WIN, total_reward: 4
[[ 0.74268912  1.01334528 -0.54069428]
 [-0.64952175  0.22006248  1.09551494]
 [ 1.21960192 -0.71201187  0.45916616]]
Total Win count: 11 || Win rate: 5.5
Total Tie count: 9 || Tie rate: 4.5
Total Lost count: 7 || Lost rate: 3.5000000000000004
Exploiting....
choose  SCISSORS
Player move : PAPER, bot: SCISSORS, reward: 1, result: WIN, total_reward: 5
[[ 0.74268912  1.01334528 -0.54069428]
 [-0.64952175  0.22006248  1.16971766]
 [ 1.21960192 -0.71201187  0.45916616]]
Total Win count: 12 || Win rate: 6.0
Total Tie count: 9 || Tie rate: 4.5
Total L